[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2001%20Accelerating%20Python%20with%20PyJulia.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>

# Fall-in-love-with-Julia: Accelerating Python 101

For Julia Python interactions there are two packages:

|    | [PyCall.jl](https://github.com/JuliaPy/PyCall.jl) | [PythonCall.jl](https://github.com/cjdoris/PythonCall.jl) |
| -: | --------- | ------------- |
| pypi | with [`PyJulia`](https://github.com/JuliaPy/pyjulia) python package (simply called `julia` on pypi) | with [`JuliaCall`](https://pypi.org/project/juliacall/) python package |
| conversions | automatically converts between native types | no auto-conversion, just wrapping |
| dependencies | Global package management via `Conda.jl` | Project-separated package management via `CondaPkg.jl` |
| run python | use `py"..."` | use `@pyexec "..."` |

Outline
- **[01](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2001%20Accelerating%20Python%20with%20PyJulia.ipynb) first notebook shows how to use PyJulia**
- [02](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2002%20Accelerating%20Python%20with%20JuliaCall.ipynb) second notebook is about JuliaCall
- [03](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2003%20Use%20Python%20with%20PyCall%20and%20PythonCall.ipynb) using Python from Julia with both PyCall and PythonCall
- [special extra](https://mybinder.org/v2/gh/jolin-io/workshop-accelerate-Python-with-Julia/main?filepath=03-example-cython-vs-cpp-vs-julia.ipynb) - Julia vs C++

# PyJulia

Let's look at the basics how to call Julia from Python with PyJulia. Note that this Jupyter has a **Python kernel**.

In [ ]:
from julia.api import Julia
_jl = Julia(compiled_modules=False)
%load_ext julia.magic

In [ ]:
import numpy as np
import pandas as pd

array = np.arange(10)
array

`PyJulia` does autoconversions for you

In [ ]:
%%julia
println(typeof($array))  # caution! @show does not work because of the special handling of interpolation $
result = isodd.($array)
println(typeof(result))

In [ ]:
result = %julia isodd.($array)
type(result)

This is especially handy when working with `True`/`False`

In [ ]:
mycondition = (array == 40).any()
mycondition

In [ ]:
%%julia
if $mycondition
    println("yes! works")
else
    println("also works :)! (we only want to test that julia actually gets a julia Bool value)")
end

## DataFrames

In [ ]:
data = {
  "calories": [420, 380, 390],
  "duration": [50, 40, 45]
}
df = pd.DataFrame(data, index = ["day1", "day2", "day3"])
df

For PyCall.jl there is not much default support for converting a Pandas Dataframe to a Julia Dataframe. Thankfully there is a separate package, compatible with PyCall, which wraps Pandas within Julia.

In [ ]:
%%julia
import Pandas
using DataFrames

DataFrame(Pandas.DataFrame($df))

The other way around works analogously

In [ ]:
%%julia
df2 = DataFrame(grp=repeat(1:2, 3), x=6:-1:1, y=4:9, z=[3:7; missing], id='a':'f')

In [ ]:
jdf2 = %julia df2
df2 = %julia Pandas.DataFrame(df2)
type(jdf2), type(df2)

In [ ]:
df2

For more comparisons between Pandas and Julia Dataframe see https://dataframes.juliadata.org/stable/man/comparisons/

## How to do it without %julia magic?

The julia's Main module allows you to access almost everything from Julia programmatically. 

In [ ]:
from julia import Main as jl

In [ ]:
jl.eval("""
broadcast_isodd(a) = isodd.(a)
""")

In [ ]:
jl.broadcast_isodd(array)

Caution! However be cautious that autoconversion, while handy, can also become tricky.

While this works

In [ ]:
%julia DataFrame(Pandas.DataFrame($df))

This does not work

In [ ]:
from julia import Pandas
jl.DataFrame(Pandas.DataFrame(df))

As a fallback you always have to define a custom julia function which you then can pass python objects to.

(Unfortunately Python does not have an interpolation syntax outside Jupyter %julia magics)

In [ ]:
jl.eval("""
pdf2jdf(df) = DataFrame(Pandas.DataFrame(df))
""")

In [ ]:
jl.pdf2jdf(df)

# Next: [02 JuliaCall](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=13%20Julia-Python20-%2002%20Accelerating%20Python%20with%20JuliaCall.ipynb)


For questions or suggestions please contact me at stephan.sahm@jolin.io

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>
